In [7]:
import os
from google.cloud import speech
import pandas as pd
import numpy as np


In [8]:
def convert(file_number):
    

    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'data/recordings/key.json'
    speech_client = speech.SpeechClient()

    # Example 1 & 2. Transcribe Local Media File 
    # File Size: < 10mbs, length < 1 minute
    '''
    config_wav = speech.RecognitionConfig(
        sample_rate_hertz=44100,
        enable_automatic_punctuation=True,
        language_code='en-NG',
        audio_channel_count=2,
        enable_word_time_offsets=True
    )
    '''
    # Example 3: Transcribing a long media file
    media_uri = f'gs://speech_to_text1234/recordings/{file_number}.wav'
    long_audi_wav = speech.RecognitionAudio(uri=media_uri)

    config_wav_enhanced = speech.RecognitionConfig(
        sample_rate_hertz=44100,
        enable_automatic_punctuation=False,
        language_code='en-US',
        use_enhanced=True,
        enable_word_time_offsets=True,
        model= "phone_call"
    )

    operation = speech_client.long_running_recognize(
        config=config_wav_enhanced,
        audio=long_audi_wav
    )
    response = operation.result(timeout=480)
    Number_words = 0
    word_list = []
    word_start_time = []
    word_end_time = []
    for result in response.results:
        alternative = result.alternatives[0]
    Number_words = 0
    for result in response.results:
            alternative = result.alternatives[0]
            print("Transcript: {}".format(alternative.transcript))
            
            for word_info in alternative.words:
                word = word_info.word
                start_time = word_info.start_time
                end_time = word_info.end_time
                word_list.append(word)
                word_start_time.append(start_time.total_seconds())
                word_end_time.append(end_time.total_seconds())
                """
                print(
                    f"Num: {Number_words}, Word: {word}, start_time: {start_time.total_seconds()}, end_time: {end_time.total_seconds()}"
                )
                """
                Number_words += 1
    df = pd.DataFrame(list(zip(word_list,word_start_time,word_end_time)), columns = ['Words',"Start_time","End_time"])
    df.to_csv(f'data/Text/{file_number}.csv')



In [9]:
for n in range(8,22):
    convert(f"00{n}")

InvalidArgument: 400 Invalid recognition 'config': bad encoding..